In [2]:
from common_module import movie_func, my_func
import insightface

import numpy as np
import cv2
import os, glob, sys
from tqdm import tqdm
from tabulate import tabulate

from face_recognition_module import sql_func

In [12]:
import yaml

with open('../enviroment.yaml', 'r') as yml:
    enviroment = yaml.safe_load(yml)['enviroment']
print(enviroment)

development


In [15]:
import yaml

with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)[enviroment]
print(config)


{'movie_path': 'C:\\Users\\yunos\\workspace\\python\\FaceRecognition\\db\\movie', 'movie_manage_path': 'C:\\Users\\yunos\\workspace\\python\\FaceRecognition\\db\\MovieManage.db', 'face_db_path': 'C:\\Users\\yunos\\workspace\\python\\FaceRecognition\\db\\split_db'}


In [6]:
mid = 25
sql = sql_func.FaceDB('db/MovieManage.db')
print(sql.GetRecords('Completes', ['*'], {'movie_id':mid}))
sql.UpdateRecords('Completes',{'movie_id':mid},{'flag_main':1})
print(sql.GetRecords('Completes', ['*'], {'movie_id':mid}))

[{'id': 25, 'movie_id': 25, 'flag_main': 9, 'flag_subject': 9, 'flag_bond': 9, 'flag_split': 9, 'created_at': '2022-10-24 19:40:44', 'updated_at': '2022-10-24 19:40:44'}]
[{'id': 25, 'movie_id': 25, 'flag_main': 1, 'flag_subject': 9, 'flag_bond': 9, 'flag_split': 9, 'created_at': '2022-10-24 19:40:44', 'updated_at': '2022-10-24 19:40:44'}]


In [13]:
from tabulate import tabulate
# ============================
# MovieManageの更新
# ============================

# MovieMangeを参照するsql
movie_manage_sql = sql_func.FaceDB(config['movie_manage_path'])
for path in glob.glob(f"{config['movie_path']}/*.mp4"):
    # Youtube動画IDを取得
    name = os.path.basename(path).replace('.mp4','')

    if movie_manage_sql.GetRecords('Movies',['id'],{'name':f"{name}"}, option={'sql_str':'LIMIT 1'}) == 0:
        # 動画を参照するmovieクラスを生成
        movie = movie_func.Movie(path)

        # 動画の総フレーム数を取得
        frame = movie.frame_count

        # 動画のfpsを取得
        fps = movie.fps

        # Moviesレコードを追加
        movie_manage_sql.InsertRecords('Movies',{'name':name,'fps':fps,'frame':frame,'path':path})

        # Completesレコードの追加
        movie_id = movie_manage_sql.GetRecords('Movies',['id'],{'name':f"{name}"})[0]['id']
        movie_manage_sql.InsertRecords('Completes',{'movie_id':movie_id, 'flag_main':9, 'flag_subject':9, 'flag_bond':9, 'flag_split':9})

records = movie_manage_sql.cursor.execute('SELECT m.id, m.name, m.path, m.fps, m.frame, c.flag_main, c.flag_subject, c.flag_bond, c.flag_split, c.created_at, c.updated_at FROM Movies m INNER JOIN Completes c ON m.id = c.movie_id ORDER BY m.id').fetchall()

print(tabulate(records, headers=['id','name','path','fps','frame','Fmain','Fsubject','Fbond','Fsplit','created_at','updated_at']))
del records

  id  name         path                          fps    frame    Fmain    Fsubject    Fbond    Fsplit  created_at           updated_at
----  -----------  ------------------------  -------  -------  -------  ----------  -------  --------  -------------------  -------------------
   1  wM73pe49JVg  db/movie\wM73pe49JVg.mp4  29.997    157309        1           1        1         1  2022-10-13 11:50:58  2022-10-13 11:50:58
   2  0wF6uUVT0ok  db/movie\0wF6uUVT0ok.mp4  29.9976   108324        1           1        1         1  2022-10-13 11:50:58  2022-10-13 11:50:58
   3  61FkT-f9cgU  db/movie\61FkT-f9cgU.mp4  30        108136        1           1        1         1  2022-10-13 11:50:58  2022-10-13 11:50:58
   4  -CBqH_bjms4  db/movie\-CBqH_bjms4.mp4  29.9962   189157        1           1        1         9  2022-10-24 19:40:43  2022-10-24 19:40:43
   5  -WRT0ZhBwGk  db/movie\-WRT0ZhBwGk.mp4  30.0048   206075        1           1        1         1  2022-10-24 19:40:43  2022-10-24 19:40:43
 

In [14]:
import copy
# ============================
# FaceDBの更新・顔認識処理
# ============================

face_analytics = insightface.app.FaceAnalysis()
face_analytics.prepare(ctx_id=0, det_size=(640,640))

# MovieMangeを参照するsql
movie_manage_sql = sql_func.FaceDB(config['movie_manage_path']) 

# MoviesとCompletesをinner joinして取得
sql_str = 'SELECT m.id, m.name, m.path, m.fps, m.frame, c.flag_main, c.flag_subject, c.flag_bond, c.flag_split FROM Movies m INNER JOIN Completes c ON m.id = c.movie_id'
movie_complete_records =  movie_manage_sql.cursor.execute(sql_str).fetchall() 
movie_complete_records = [{'id':r[0], 'name':r[1], 'path':r[2], 'fps':r[3], 'frame':r[4], 'flag_main':r[5], 'flag_subject':r[6], 'flag_bond':r[7], 'flag_split':r[8]} for r in movie_complete_records]

# 顔認識処理を行うframe rate
frame_rate = 10
for mcr in movie_complete_records:
    # 顔認識処理
    if mcr['flag_main'] == 9: # 顔認識処理が実行されていない
        print(mcr)
        print(my_func.seconds2time(mcr['frame']/mcr['fps']))
        face_db_sql = sql_func.FaceDB(f"{config['face_db_path']}/FaceDB{mcr['id']}.db") # FaceDBを参照するsql
        face_db_mrs = face_db_sql.GetRecords('Movies',['*']) # FaceDBのMoviesレコードを取得

        # FaceDBにおけるMoviesレコードの存在チェック
        if len(face_db_mrs) > 0: # FaceDBのMoviesにレコードが存在する
            face_db_mr = face_db_mrs[0]

            # movie manageの情報とFaceDbの情報が一致しているかチェック
            if mcr['name'] != face_db_mr['name']:
                print('MovieManageとFaceDBが一致していません. id:',mcr['id'])
                break
        else: # FaceDBのMoviesにレコードが存在しない
            # FaceDBのMoviesにMovieManageのMoviesレコードを追加
            movie_manage_mr = movie_manage_sql.GetRecords('Movies',['*'],{'id':mcr['id']})[0]
            face_db_sql.InsertRecords('Movies',movie_manage_mr)

            # FaceDBのMoviesレコードを取得
            face_db_mr = face_db_sql.GetRecords('Movies',['*'])[0]
        
        movie_manage_cr = movie_manage_sql.GetRecords('Completes',['*'],{'movie_id':mcr['id']},option={'sql_str':'LIMIT 1'})[0]
        face_db_sql.UpdateRecords('Completes', {'movie_id':mcr['id']}, movie_manage_cr)

        #
        movie = movie_func.Movie(f"{config['movie_path']}/{face_db_mr['name']}.mp4")
        # Facesテーブルのframeカラムの最大値を取得（前回最後に読み込んフレーム値を取得）
        last_frame_records = face_db_sql.GetRecords('Faces',['frame'],option={'sql_str':'ORDER BY FRAME DESC LIMIT 1'})
        if len(last_frame_records) > 0:
            last_frame = last_frame_records[0]['frame']
            sql_str = f'DELETE FROM Faces WHERE frame >= {last_frame}'
            face_db_sql.cursor.execute(sql_str)
            face_db_sql.connect.commit()
        else:
            last_frame = 1
        progressbar = tqdm(np.arange(last_frame, face_db_mr['frame']+1,frame_rate),ncols=0)
        movie_id = face_db_mr['id']
        progressbar.set_description(f"movie_id:{movie_id}")

        insert_values = []
        for frame in progressbar:
            ret, img = movie.GetImage(frame)
            if ret:
                faces = face_analytics.get(np.asarray(img))
                for face in faces:
                    for k in face.keys():
                        if k in ['bbox','kps','landmark_3d_68','pose','landmark_2d_106','embedding']:
                            face[k] = face[k].tolist()
                        elif k in ['det_score']:
                            face[k] = float(face[k])
                        elif k in ['gender','age']:
                            face[k] = int(face[k])
                    face['movie_id'] = int(movie_id)
                    face['frame'] = int(frame)
                insert_values += faces
            if len(insert_values) > 5000:
                face_db_sql.BulkInsertRecords('Faces',insert_values)
                insert_values = []
        
        face_db_sql.UpdateRecords('Completes',{'movie_id':movie_id}, {'flag_main':1})
        face_db_cr = face_db_sql.GetRecords('Completes',{'movie_id':movie_id})[0]
        movie_manage_sql.UpdateRecords('Completes',{'movie_id':movie_id},face_db_cr)
        break

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\yunos/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\yunos/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\yunos/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\yunos/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\yunos/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

movie_id:26:   0% 8/9819 [00:00<18:00,  9.08it/s]c:\Users\yunos\workspace\python\FaceRecognition\.venv\lib\site-packages\insightface\utils\transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
movie_id:26:   1% 57/9819 [00:07<22:11,  7.33it/s]


KeyboardInterrupt: 

In [5]:
face_db_sql = sql_func.FaceDB(f"db/split_db/FaceDB25.db")
face_db_sql.GetRecords('Faces',['id','frame','created_at','updated_at'],option={'sql_str':'ORDER BY frame DESC LIMIT 100'})

[{'id': 5231088,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231089,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231090,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231091,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231092,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231093,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231094,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231095,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at': '2022-12-22 18:06:24'},
 {'id': 5231096,
  'frame': 55761,
  'created_at': '2022-12-22 18:06:24',
  'updated_at'